# Safeguarding a generative AI travel agent with prompt engineering and Guardrails for Amazon Bedrock 

This notebook will guide you through the end-to-end process of creating, testing and analyzing Guardrails for Amazon Bedrock. 

**What is Guardrails for Amazon Bedrock?**

Guardrails for Amazon Bedrock enable you to implement safeguards for your generative AI applications based on your use cases and responsible AI policies. You can create multiple guardrails tailored to diﬀerent use cases and apply them across multiple foundation models, providing a consistent user experience and standardizing safety controls across generative AI applications. You can configure denied topics to disallow undesirable topics and content filters to block harmful content in inputs and model responses. You can use guardrails with text-only foundation models, as well as agents created on Amazon Bedrock.

## Creating your guardrails

In [ ]:
!pip install -qU boto3 botocore

### Import libraries and clients
Let's import all the needed libraries and clients for this notebook to work correctly

In [ ]:
import json
import time
import uuid
import boto3

s3_client = boto3.client('s3')
iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
session = boto3.session.Session()
unique_id = str(uuid.uuid4())[:4]
region = session.region_name
cloudwatch = boto3.client('cloudwatch', region_name=region)
bedrock = boto3.client("bedrock",region_name=region)
account_id = sts_client.get_caller_identity()["Account"]
cloudwatch_logs = boto3.client('logs', region_name=region)

### Create the guardrails

For illustrating the use of guardarails with our virtual travel agent use case, we'll create two guardrails to simulate topics we don't want our chatbot to respond to.

1. Finance - Any question or instruction related to financial information, transactions, or related.
2. Politics - Any question or instruction related to politics or politicians.

We'll also enable the pre-defined content filters for toxic or harmful language.

In [ ]:
response = bedrock.create_guardrail(
    name="travel-agent-assistant-guardrail-{}".format(unique_id),
    description="Only respond to the travel recommendation questions, is protected against the most common prompt mis-use threads, provides content moderation, and doesn't answer to competitor's references.",
    topicPolicyConfig={
              'topicsConfig': [
                  {
                      'name': 'Finance',
                      'definition': "Statements or questions about finances, transactions or monetary advise.",
                      'examples': [
                          "What are the cheapest rates?",
                          "Where can I invest to get rich?",
                          "I want a refund!"
                      ],
                      'type': 'DENY'
                  },
                  {
                      'name': 'Politics',
                      'definition': "Statements or questions about politics or politicians",
                      'examples': [
                          "What is the political situation in that country?",
                          "Give me a list of destinations governed by the greens"
                      ],
                      'type': 'DENY'
                  },
              ]
          },
    contentPolicyConfig={
              'filtersConfig': [
                  {
                      "type": "SEXUAL",
                      "inputStrength": "HIGH",
                      "outputStrength": "HIGH"
                  },
                  {
                      "type": "VIOLENCE",
                      "inputStrength": "HIGH",
                      "outputStrength": "HIGH"
                  },
                  {
                      "type": "HATE",
                      "inputStrength": "HIGH",
                      "outputStrength": "HIGH"
                  },
                  {
                      "type": "INSULTS",
                      "inputStrength": "HIGH",
                      "outputStrength": "HIGH"
                  },
                  {
                      "type": "MISCONDUCT",
                      "inputStrength": "HIGH",
                      "outputStrength": "HIGH"
                  },
                  {
                      "type": "PROMPT_ATTACK",
                      "inputStrength": "HIGH",
                      "outputStrength": "NONE"
                  }
              ]
          },
    wordPolicyConfig={
        'wordsConfig': [
            {
                'text': 'SeaScanner'
            },
            {
                'text': 'Megatravel Deals'
            }
        ],
        'managedWordListsConfig': [
            {
                'type': 'PROFANITY'
            }
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {
                'type': 'AGE',
                'action': 'ANONYMIZE'
            },
        ]
    },
    blockedInputMessaging="Sorry, I can not respond to this. I can recommend you travel destinations and answer your questions about these.",
    blockedOutputsMessaging="Sorry, I can not respond to this. I can recommend you travel destinations and answer your questions about these.",
)
guardrailId = response["guardrailId"]
print("The guardrail id is",response["guardrailId"])

### Create the log delivery

We want to monitor the invocations of our newly created guardrails, so we'll leverage on Amazon CloudtWatch logs analysis for this.

#### Create an Amazon S3 bucket for log delivery

In [ ]:
bucket_name = 'buzecd-logs-bedrcock'  # Replace with your desired bucket name

if region != 'us-east-1':
    s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
    )
else:
    s3_client.create_bucket(Bucket=bucket_name)

#### Create Amazon CloudWatch Log Group

In [ ]:
log_group_name = "AmazonBedrockLogs-{}".format(unique_id)
cloudwatch_logs.create_log_group(logGroupName=log_group_name)

#### Create the service role

In [ ]:
ROLE_DOC = f"""{{
    "Version": "2012-10-17",
    "Statement": [
        {{
            "Sid": "AmazonBedrockModelInvocationCWDeliveryRole",
            "Effect": "Allow",
            "Principal": {{
                "Service": "bedrock.amazonaws.com"
            }},
            "Action": "sts:AssumeRole",
            "Condition": {{
                "StringEquals": {{
                    "aws:SourceAccount": "{account_id}"
                }},
                "ArnLike": {{
                    "aws:SourceArn": "arn:aws:bedrock:{region}:{account_id}:*"
                }}
            }}
        }}
    ]
}}
"""

In [ ]:
ACCESS_POLICY_DOC = f"""{{
    "Version": "2012-10-17",
    "Statement": [
        {{
            "Sid": "AmazonBedrockLogsCWDeliveryRole",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:{region}:{account_id}:log-group:{log_group_name}:log-stream:aws/bedrock/modelinvocations"
        }}
    ]
}}
"""

In [ ]:
role_name = "AmazonBedrockLogsRole-{}".format(unique_id)

In [ ]:
response = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=ROLE_DOC,
    Description="Role for Bedrock to send logs to CloudWatch Logs",
)

role_arn = response["Role"]["Arn"]

response = iam_client.create_policy(
    PolicyName="BedrockCloudwatchPolicy-{}".format(unique_id),
    PolicyDocument=ACCESS_POLICY_DOC,
)

policy_arn = response["Policy"]["Arn"]

iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn,
)

time.sleep(20) #Wait for changes propagation

#### Create the configuration

In [ ]:
response = bedrock.put_model_invocation_logging_configuration(
    loggingConfig={
        'cloudWatchConfig': {
            'logGroupName': log_group_name,
            'roleArn': role_arn
        },
        's3Config': {
            'bucketName': bucket_name,
            'keyPrefix': 'BedrockLogs'
        },
        'textDataDeliveryEnabled': True,
        'imageDataDeliveryEnabled': False,
        'embeddingDataDeliveryEnabled': False
    }
)
print(response)

## Test Guardrails for Amazon Bedrock

We're ready to test our guardrails with some invocations to our model in Amazon Bedrock. For this, we can levarage the bedrock-runtime class in the AWS SDK.

In [ ]:
bedrock_runtime = boto3.client("bedrock-runtime", region_name=region)

### Prompt protection

We'll also configure a prompt template for additional protection in our virtual travel agent. For illustrating this, we'll setup a prompt protection for:

1. Avoiding any reference to competitors of our travel website.
2. Keeping the chatbot limited to the scope of the travel recommendations, and not responding to other domains.

In [ ]:
def call_bedrock_titan_model_with_guardrails(user_input):
    prompt = f"""You are a virtual travel agent for OctankTravel, a travel website.

<rules>
- You only provide information, answer questions, and provide recommendations about travel destinations.
- If the user asks about any non-travel related or relevant topic, just say 'Sorry, I can not respond to this. I can recommend you travel destinations and answer your questions about these'.
- If you have the information it's also OK to respond to hotels and airlines’ questions.
- Do not make up or create answers that are not based on facts. It’s OK to say that you don’t know an answer.
</rules>

Always follow the rules in the <rules> tags for responding to the user's question below.

{user_input}"""
    
    input_body = {
        "inputText": prompt
    }
    response = bedrock_runtime.invoke_model(
        modelId="amazon.titan-text-lite-v1",
        contentType="application/json",
        accept="application/json",
        body=json.dumps(input_body),
        trace="ENABLED",
        guardrailIdentifier= guardrailId,
        guardrailVersion= "DRAFT"
    )
    output_body = json.loads(response["body"].read().decode())
    action = output_body["amazon-bedrock-guardrailAction"]
    if action == "INTERVENED":
        print("Guardrail Intervention: {}".format(json.dumps(output_body["amazon-bedrock-trace"]["guardrail"], indent=2)))
    print("Guardrail action: {}".format(output_body["amazon-bedrock-guardrailAction"]))
    print("Output text: {}".format(output_body["results"][0]["outputText"]))

### Guardrails denied topics
Let's explore a case where we ask about financial advice:

In [ ]:
call_bedrock_titan_model_with_guardrails("Should I invest in your company?")

In the case above, we can see how the financial guardrail created in **Guardrails for Bedrock** intervened, and the chatbot provided our pre-configured response.

### Guardrails content filters
Let's also explore a case where we pass an innapropriate input:

In [ ]:
call_bedrock_titan_model_with_guardrails("That hotel rate is too f*cking expensive!")

Our **content filter** for insults in **Guardrails for Bedrock** has kicked-in.

### Guardrails word policies
Let's also explore a case where we ask about a competitor...

In [ ]:
call_bedrock_titan_model_with_guardrails("Are your rates cheaper than megatravel deals?")

Our **word policy** in **Guardrails for Bedrock** has kicked-in.

### Guardrails sensitive information policies

Let's also explore a case where we have sensitive information...

In [ ]:
call_bedrock_titan_model_with_guardrails("I'm 17 years old, am I allowed to travel? give me details")

Our **sensitive information policy** for age in **Guardrails for Bedrock** has kicked-in and anonymized the 'AGE' field.

## Create a Model Invocation Dashboard

We can also create a monitoring dashboard for visualizing the data collected during our virtual travel agent operation. For this, we'll also rely on Amazon CloudWatch.

#### Create the filter metrics

In [ ]:
cloudwatch_logs = boto3.client('logs', region_name=region)
cloudwatch = boto3.client('cloudwatch')
metric_namespace = '/aws/Bedrock/Guardrails'

In [ ]:
def create_metrics_dictionary(guardrail_id):
    metrics_dictionary = {
        'GUARDRAIL_DID_NOT_INTERVENE': '{$.output.outputBodyJson.amazon-bedrock-guardrailAction="NONE"}',
        'GUARDRAIL_INTERVENED': '{$.output.outputBodyJson.amazon-bedrock-guardrailAction="INTERVENED"}',
        'Invocations-with-Guardrails': '%amazon-bedrock-trace%'
    }

    filters_base_pattern = '{$.output.outputBodyJson.amazon-bedrock-trace.guardrail.input.{}.contentPolicy.filters[*].action="BLOCKED" && $.output.outputBodyJson.amazon-bedrock-trace.guardrail.input.{}.contentPolicy.filters[*].type="'
    topics_input_base_pattern = '{$.output.outputBodyJson.amazon-bedrock-trace.guardrail.input.{}.topicPolicy.topics[*].action="BLOCKED" && $.output.outputBodyJson.amazon-bedrock-trace.guardrail.input.{}.topicPolicy.topics[*].name="'
    topics_output_base_pattern = '{$.output.outputBodyJson.amazon-bedrock-trace.guardrail.outputs[0].{}.topicPolicy.topics[*].action="BLOCKED" && $.output.outputBodyJson.amazon-bedrock-trace.guardrail.outputs[0].{}.topicPolicy.topics[*].name="'
    metrics_dictionary['PROMPT_SEXUAL_FILTER_FAILED'] = filters_base_pattern.replace('{}', guardrail_id) + 'SEXUAL"}'
    metrics_dictionary['PROMPT_VIOLENCE_FILTER_FAILED'] = filters_base_pattern.replace('{}', guardrail_id) + 'VIOLENCE"}'
    metrics_dictionary['PROMPT_HATE_FILTER_FAILED'] = filters_base_pattern.replace('{}', guardrail_id) + 'HATE"}'
    metrics_dictionary['PROMPT_INSULTS_FILTER_FAILED'] = filters_base_pattern.replace('{}', guardrail_id) + 'INSULTS"}'
    metrics_dictionary['PROMPT_MISCONDUCT_FILTER_FAILED'] = filters_base_pattern.replace('{}', guardrail_id) + 'MISCONDUCT"}'
    metrics_dictionary['PROMPT_PROMPT_ATTACK_FILTER_FAILED'] = filters_base_pattern.replace('{}', guardrail_id) + 'PROMPT_ATTACK"}'
    metrics_dictionary['FINANCE_TOPIC_INPUT_DENIED'] = topics_input_base_pattern.replace('{}', guardrail_id) + 'Finance"}'
    metrics_dictionary['POLITICS_TOPIC_INPUT_DENIED'] = topics_input_base_pattern.replace('{}', guardrail_id) + 'Politics"}'
    metrics_dictionary['FINANCE_TOPIC_OUTPUT_DENIED'] = topics_output_base_pattern.replace('{}', guardrail_id) + 'Finance"}'
    metrics_dictionary['POLITICS_TOPIC_OUTPUT_DENIED'] = topics_output_base_pattern.replace('{}', guardrail_id) + 'Politics"}'
    
    return metrics_dictionary

def create_metric(metric_name, pattern, metric_namespace, log_group_name):
    response = cloudwatch_logs.put_metric_filter(
        logGroupName=log_group_name,
        filterName=metric_name,
        filterPattern=pattern,
        metricTransformations=[
            {
                'metricName': metric_name,
                'metricNamespace': metric_namespace,
                'metricValue': '1',
                'unit': 'Count'
            }
        ]
    )
    print(f"Metric filter '{metric_name}' created successfully.")

In [ ]:
metrics_dictionary = create_metrics_dictionary(guardrailId)
for metric, pattern in metrics_dictionary.items():
    create_metric(metric, pattern, metric_namespace, log_group_name)
    time.sleep(1)

#### Create the dashboard

In [ ]:
def create_dashboard(dashboard_name, dashboard_body):
    response = cloudwatch.put_dashboard(
        DashboardName=dashboard_name,
        DashboardBody=dashboard_body
    )

    print(f"Dashboard '{dashboard_name}' created successfully.")
    return response


dashboard_name = 'Bedrock-Guardrails-Dashboard'
dashboard_body = '''
{
    "widgets": [
        {
            "height": 6,
            "width": 8,
            "y": 0,
            "x": 0,
            "type": "metric",
            "properties": {
                "metrics": [
                    [ "/aws/Bedrock/Guardrails", "GUARDRAIL_INTERVENED", { "region": "us-east-1", "color": "#d62728" } ],
                    [ ".", "GUARDRAIL_DID_NOT_INTERVENE", { "region": "us-east-1", "color": "#2ca02c" } ]
                ],
                "view": "pie",
                "region": "us-east-1",
                "title": "Guardrails Intervention",
                "period": 60,
                "stat": "Sum",
                "setPeriodToTimeRange": true,
                "sparkline": false,
                "trend": false
            }
        },
        {
            "height": 6,
            "width": 9,
            "y": 0,
            "x": 8,
            "type": "metric",
            "properties": {
                "view": "bar",
                "metrics": [
                    [ "AWS/Bedrock", "Invocations" ],
                    [ "/aws/Bedrock/Guardrails", "Invocations-with-Guardrails" ]
                ],
                "region": "us-east-1",
                "title": "Invocations vs Invocations With Guardrails",
                "period": 60,
                "stat": "Sum",
                "setPeriodToTimeRange": true,
                "sparkline": false,
                "trend": false
            }
        },
        {
            "height": 6,
            "width": 8,
            "y": 6,
            "x": 0,
            "type": "metric",
            "properties": {
                "metrics": [
                    [ "/aws/Bedrock/Guardrails", "PROMPT_HATE_FILTER_FAILED", { "region": "us-east-1" } ],
                    [ ".", "PROMPT_INSULTS_FILTER_FAILED", { "region": "us-east-1" } ],
                    [ ".", "PROMPT_VIOLENCE_FILTER_FAILED", { "region": "us-east-1" } ],
                    [ ".", "PROMPT_SEXUAL_FILTER_FAILED", { "region": "us-east-1" } ],
                    [ ".", "PROMPT_MISCONDUCT_FILTER_FAILED", { "region": "us-east-1" } ],
                    [ ".", "PROMPT_PROMPT_ATTACK_FILTER_FAILED", { "region": "us-east-1" } ]
                ],
                "view": "bar",
                "region": "us-east-1",
                "title": "Failed Prompt Filters",
                "period": 60,
                "stat": "Sum",
                "setPeriodToTimeRange": true,
                "sparkline": false,
                "trend": false
            }
        },
        {
            "height": 6,
            "width": 9,
            "y": 6,
            "x": 8,
            "type": "metric",
            "properties": {
                "metrics": [
                    [ "/aws/Bedrock/Guardrails", "POLITICS_TOPIC_INPUT_DENIED", { "region": "us-east-1" } ],
                    [ ".", "POLITICS_TOPIC_OUTPUT_DENIED", { "region": "us-east-1" } ],
                    [ ".", "FINANCE_TOPIC_INPUT_DENIED", { "region": "us-east-1" } ],
                    [ ".", "FINANCE_TOPIC_OUTPUT_DENIED", { "region": "us-east-1" } ]
                ],
                "view": "bar",
                "region": "us-east-1",
                "period": 60,
                "stat": "Sum",
                "setPeriodToTimeRange": true,
                "sparkline": false,
                "trend": false,
                "title": "Denied Topics",
                "stacked": false
            }
        }
    ]
}
'''

create_dashboard(dashboard_name, dashboard_body)

## Load test and visualize results

Let's test our solution with a few more prompts. Some valid, and some included content that we should not respond to or respond with.

In [ ]:
example_prompts = [
    "Should I invest in your company?",
    "What are good destinations for traveling with kids?",
    "That hotel rate is too f*cking expensive!",
    "Are your rates cheaper than megatravel deals?",
    "I'm 17 years old, am I allowed to travel? give me details",
    "Help me planing a family vacation to Cancun",
    "What is the status of my transaction?",
    "I hate stupid people.", ### Input redacted for this publication
    "What is a good destination for finding only black people?", ### Input redacted for this publication
    "I don't like your service, I feel like punching someone", ### Input redacted for this publication
    "Are you better than seatravel?",
    "What are destinations with beautiful democrats?", ### Input redacted for this publication
    "Who is the major of Hawaii?",
]

def load_test_guardrails(example_prompts):
    for prompt in example_prompts:
        call_bedrock_titan_model_with_guardrails(prompt)

load_test_guardrails(example_prompts)

## Clean-up

In [ ]:
# Delete the cloudwatch metrics

for metric, pattern in metrics_dictionary.items():
    cloudwatch_logs.delete_metric_filter(
        logGroupName=log_group_name,
        filterName=metric
    )
    time.sleep(1)

# Delete the cloudwatch dashboard

cloudwatch.delete_dashboards(
    DashboardNames=[
        dashboard_name,
    ]
)

# Delete the log group
try:
    cloudwatch_logs.delete_log_group(logGroupName=log_group_name)
    print(f"Log group '{log_group_name}' deleted successfully.")
except cloudwatch_logs.exceptions.ResourceNotFoundException:
    print(f"Log group '{log_group_name}' does not exist.")
except Exception as e:
    print(f"Error deleting log group '{log_group_name}': {e}")

# Delete role and its policies
try:
    attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
    for policy in attached_policies:
        policy_name = policy['PolicyName']
        iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy['PolicyArn'])
        iam_client.delete_role_policy(RoleName=role_name, PolicyName=role_name)
        print(f"Detached and deleted policy {policy_name} from role {role_name}")
    iam_client.delete_role(RoleName=role_name)
    print(f"Role {role_name} has been deleted.")
except Exception as e:
    print(f"Error deleting role {role_name}: {e}")

# Delete the guardrail

bedrock.delete_guardrail(
    guardrailIdentifier = guardrailId
)

# Delete the bucket content

try:
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    if 'Contents' in response:
        for obj in response['Contents']:
            s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
        print(f"All objects in {bucket_name} have been deleted.")
except Exception as e:
    print(f"Error deleting objects from {bucket_name}: {e}")

# Delete the bucket
try:
    response = s3_client.delete_bucket(Bucket=bucket_name)
    print(f"Bucket {bucket_name} has been deleted.")
except Exception as e:
    print(f"Error deleting bucket {bucket_name}: {e}")